In [30]:
import pandas as pd
import sqlite3
import os

conn = sqlite3.connect(":memory:")
data_dir = "./Store_Data"

# Loop through all .csv files in the directory
for file in os.listdir(data_dir):
    if file.endswith(".csv"):
        table_name = os.path.splitext(file)[0]  # remove .csv extension
        df = pd.read_csv(os.path.join(data_dir, file))
        df.to_sql(table_name, conn, index=False, if_exists="replace")


In [31]:
# Q1: Find the most senior employee based on job title.
# Hint: Use the employee table and sort by the levels column in descending order.


# Run SQL query
result = pd.read_sql("SELECT first_name, last_name, levels FROM employee ORDER BY levels DESC", conn)
print(result)

  first_name last_name levels
0      Mohan     Madan     L7
1     Andrew     Adams     L6
2      Nancy   Edwards     L4
3    Michael  Mitchell     L3
4     Robert      King     L2
5      Laura  Callahan     L2
6       Jane   Peacock     L1
7   Margaret      Park     L1
8      Steve   Johnson     L1


In [32]:
# Q2: Determine which countries have the most invoices.
# Hint: Group the invoice data by billing_country, then count and sort the results.

# Run SQL query
result = pd.read_sql("SELECT billing_country, COUNT(*) AS invoice_count FROM invoice GROUP BY billing_country ORDER BY invoice_count DESC", conn)
print(result)

   billing_country  invoice_count
0              USA            131
1           Canada             76
2           Brazil             61
3           France             50
4          Germany             41
5   Czech Republic             30
6         Portugal             29
7   United Kingdom             28
8            India             21
9          Ireland             13
10           Chile             13
11           Spain             11
12         Finland             11
13          Sweden             10
14          Poland             10
15     Netherlands             10
16         Hungary             10
17         Denmark             10
18       Australia             10
19          Norway              9
20           Italy              9
21         Austria              9
22         Belgium              7
23       Argentina              5


In [33]:
# Q3: Identify the top 3 invoice totals.
# Hint: Sort the invoice table by the total column.

total = pd.read_sql("SELECT sum(total) AS Total FROM invoice ORDER BY total DESC LIMIT 3", conn)
print(total)

     Total
0  4709.43


In [34]:
# Q4: Find the city with the highest total invoice amount to determine the best location for
# a promotional event.

total = pd.read_sql("SELECT billing_city AS city, sum(total) AS Total FROM invoice ORDER BY total DESC LIMIT 3", conn)
print(total)


       city    Total
0  New York  4709.43


In [35]:
# Q5: Identify the customer who has spent the most money.
# Hint: Use a join between customer and invoice, group by customer_id, and sum
# the totals.

customer = pd.read_sql("""
SELECT 
    c.first_name, 
    c.last_name, 
    SUM(i.total) AS FinalTotal
FROM customer AS c
INNER JOIN invoice AS i ON c.customer_id = i.customer_id
GROUP BY c.customer_id
ORDER BY FinalTotal DESC
LIMIT 1
""", conn)

print(customer)


  first_name    last_name  FinalTotal
0  František  Wichterlová      144.54


In [36]:
# Q1: Find the email, first name, and last name of customers who listen to Rock music.
# Hint: Use joins across customer, invoice, invoice_line, and track, filtering for the
# genre Rock.

music = pd.read_sql("""
SELECT DISTINCT
    c.first_name, 
    c.last_name, 
    c.email 
FROM customer AS c
INNER JOIN invoice AS i ON c.customer_id = i.customer_id
INNER JOIN invoice_line AS il ON i.invoice_id = il.invoice_id
INNER JOIN track AS t ON il.track_id = t.track_id
INNER JOIN genre AS g ON t.genre_id = g.genre_id
WHERE g.name = 'Rock';
""", conn)

print(music)

   first_name     last_name                          email
0        Phil        Hughes          phil.hughes@gmail.com
1       Aaron      Mitchell         aaronmitchell@yahoo.ca
2       Terhi    Hämäläinen      terhi.hamalainen@apple.fi
3       Wyatt        Girard          wyatt.girard@yahoo.fr
4       Steve        Murray          steve.murray@yahoo.uk
5    Johannes  Van der Berg        johavanderberg@yahoo.nl
6   Stanisław        Wójcik         stanisław.wójcik@wp.pl
7       Kathy         Chase            kachase@hotmail.com
8    Madalena       Sampaio              masampaio@sapo.pt
9        Jack         Smith        jacksmith@microsoft.com
10     Helena          Holý                hholy@gmail.com
11      Frank       Ralston             fralston@gmail.com
12     Edward       Francis            edfrancis@yachoo.ca
13       João     Fernandes            jfernandes@yahoo.pt
14     Martha          Silk           marthasilk@gmail.com
15     Niklas      Schröder            nschroder@surfeu.

In [18]:
# Q2: Identify the top 10 rock artists based on track count.
# Hint: Use joins across artist, album, and track and filter by the genre Rock.
# Count tracks per artist.

music = pd.read_sql("""
SELECT 
    a.name AS artist_name,
    COUNT(t.track_id) AS rock_track_count
FROM artist AS a
INNER JOIN (
    SELECT * FROM album
    UNION
    SELECT * FROM album2
) AS combined_album ON a.artist_id = combined_album.artist_id
INNER JOIN track AS t ON combined_album.album_id = t.album_id
INNER JOIN genre AS g ON t.genre_id = g.genre_id
WHERE g.name = 'Rock'
GROUP BY a.artist_id, a.name
ORDER BY rock_track_count DESC
LIMIT 10;
""", conn)

print(music)

                    artist_name  rock_track_count
0                  Led Zeppelin               161
1                            U2               152
2                   Deep Purple               110
3                   Iron Maiden                97
4                     Pearl Jam                82
5  Creedence Clearwater Revival                80
6                         Queen                79
7                     Van Halen                69
8                  Foo Fighters                53
9            The Rolling Stones                53


In [37]:
# Q3: Find all track names that are longer than the average track length.
# Hint: Calculate the average length and compare each track’s length to this
# average.

tracklengh = pd.read_sql("""
SELECT name, milliseconds
FROM track
WHERE milliseconds > (
SELECT avg(milliseconds) FROM track
)
""", conn)

print(tracklengh)


                                                  name  milliseconds
0                          You Oughta Know (Alternate)        491885
1                                    Master Of Puppets        436453
2                            Snoopy's search-Red baron        456071
3                                              Stratus        582086
4                                        No More Tears        555075
..                                                 ...           ...
489                             Rehab (Hot Chip Remix)        418293
490                        You Sent Me Flying / Cherry        409906
491                                Amy Amy Amy (Outro)        663426
492  Symphony No. 3 Op. 36 for Orchestra and Sopran...        567494
493  Concerto for Violin, Strings and Continuo in G...        493573

[494 rows x 2 columns]


In [38]:
# Q1: Calculate how much each customer has spent on each artist.
# Hint: Use a CTE to calculate the earnings per artist from invoice_line, then join it
# with customer, invoice, and artist.

spent = pd.read_sql("""
WITH artist_revenue AS (
    SELECT 
        il.invoice_id,
        ar.artist_id,
        ar.name AS artist_name,
        il.unit_price * il.quantity AS amount
    FROM invoice_line il
    JOIN track t ON il.track_id = t.track_id
    JOIN album al ON t.album_id = al.album_id
    JOIN artist ar ON al.artist_id = ar.artist_id
)
SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    ar.artist_name,
    ROUND(SUM(ar.amount), 2) AS total_spent
FROM artist_revenue ar
JOIN invoice i ON ar.invoice_id = i.invoice_id
JOIN customer c ON i.customer_id = c.customer_id
GROUP BY c.customer_id, ar.artist_id
ORDER BY total_spent DESC;
""", conn)

print(spent)

      customer_id first_name    last_name            artist_name  total_spent
0              46       Hugh     O'Reilly                  Queen        27.72
1              42      Wyatt       Girard          Frank Sinatra        23.76
2               3   François     Tremblay                The Who        19.80
3               5  František  Wichterlová                   Kiss        19.80
4               6     Helena         Holý  Red Hot Chili Peppers        19.80
...           ...        ...          ...                    ...          ...
2184           59    Rishabh       Mishra              Van Halen         0.99
2185           59    Rishabh       Mishra              Scorpions         0.99
2186           59    Rishabh       Mishra                   Cake         0.99
2187           59    Rishabh       Mishra    Christopher O'Riley         0.99
2188           59    Rishabh       Mishra          Amy Winehouse         0.99

[2189 rows x 5 columns]


In [39]:
# Q2: Determine the most popular music genre for each country based on purchases.
# Hint: Use a CTE or window function to rank genres by purchase count per
# country.

musicgenre = pd.read_sql("""
WITH musicgenre AS (
    SELECT 
        c.country,
        g.name AS genre_name,
        COUNT(*) AS purchase_count
    FROM customer AS c
    JOIN invoice AS i ON c.customer_id = i.customer_id
    JOIN invoice_line AS il ON i.invoice_id = il.invoice_id
    JOIN track AS t ON il.track_id = t.track_id
    JOIN genre AS g ON t.genre_id = g.genre_id
    GROUP BY c.country, g.name
),
ranked_genres AS (
    SELECT 
        country,  
        genre_name,  
        purchase_count,
        ROW_NUMBER() OVER (PARTITION BY country ORDER BY purchase_count DESC) AS genre_rank
    FROM musicgenre 
)
SELECT 
    country,  
    genre_name AS most_popular_genre,
    purchase_count
FROM ranked_genres
WHERE genre_rank = 1
ORDER BY country;
""", conn)

print(musicgenre)

           country  most_popular_genre  purchase_count
0        Argentina  Alternative & Punk              17
1        Australia                Rock              34
2          Austria                Rock              40
3          Belgium                Rock              26
4           Brazil                Rock             205
5           Canada                Rock             333
6            Chile                Rock              61
7   Czech Republic                Rock             143
8          Denmark                Rock              24
9          Finland                Rock              46
10          France                Rock             211
11         Germany                Rock             194
12         Hungary                Rock              44
13           India                Rock             102
14         Ireland                Rock              72
15           Italy                Rock              35
16     Netherlands                Rock              33
17        

In [40]:
# Q3: Identify the top-spending customer for each country.
# Hint: Calculate the total spending per customer per country and filter for the
# highest spending.

top_customers = pd.read_sql("""
SELECT 
    cs.customer_id,
    cs.first_name,
    cs.last_name,
    cs.country,
    ROUND(cs.total_spent, 2) AS total_spent
FROM (
    SELECT 
        c.customer_id,
        c.first_name,
        c.last_name,
        c.country,
        SUM(il.unit_price * il.quantity) AS total_spent,
        ROW_NUMBER() OVER (PARTITION BY c.country ORDER BY SUM(il.unit_price * il.quantity) DESC) AS rank
    FROM customer AS c
    JOIN invoice AS i ON c.customer_id = i.customer_id
    JOIN invoice_line AS il ON i.invoice_id = il.invoice_id
    GROUP BY c.customer_id, c.first_name, c.last_name, c.country
) cs
WHERE cs.rank = 1
ORDER BY cs.country;
""", conn)

print(top_customers)

    customer_id first_name     last_name         country  total_spent
0            56      Diego     Gutiérrez       Argentina        39.60
1            55       Mark        Taylor       Australia        81.18
2             7     Astrid        Gruber         Austria        69.30
3             8       Daan       Peeters         Belgium        60.39
4             1       Luís     Gonçalves          Brazil       108.90
5             3   François      Tremblay          Canada        99.99
6            57       Luis         Rojas           Chile        97.02
7             5  František   Wichterlová  Czech Republic       144.54
8             9       Kara       Nielsen         Denmark        37.62
9            44      Terhi    Hämäläinen         Finland        79.20
10           42      Wyatt        Girard          France        99.99
11           37       Fynn    Zimmermann         Germany        94.05
12           45   Ladislav        Kovács         Hungary        78.21
13           58     